In [1]:
from threading import local, get_ident
from queue import Queue

QUEUE = Queue()


class ContextTracker:
    def __init__(self):
        self.contexts = []

    def call_with_context(self, newContext, func, *args, **kwargs):
        self.contexts.append(newContext)
        try:
            return func(*args, **kwargs)
        finally:
            self.contexts.pop()

    def get_context(self, key, default=None):
        QUEUE.put(f"{get_ident()}, {self.contexts}")
        for ctx in reversed(self.contexts):
            try:
                return ctx[key]
            except KeyError:
                pass
        return default


class ThreadedContextTracker:
    def __init__(self):
        self.storage = local()

    def current_context(self):
        try:
            return self.storage.ct
        except AttributeError:
            ct = self.storage.ct = ContextTracker()
            return ct

    def call_with_context(self, ctx, func, *args, **kw):
        return self.current_context().call_with_context(ctx, func, *args, **kw)

    def get_context(self, key, default=None):
        return self.current_context().get_context(key, default)


class RequestContext:
    KEY = "X_RAY_CONTEXT"

    def __init__(self):
        self.context = ThreadedContextTracker()

    def call_with_context(self, ctx, func, *args, **kwargs):
        return self.context.call_with_context({RequestContext.KEY: ctx}, func, *args, **kwargs)
    
    def get_context(self, default=None):
        return self.context.get_context(RequestContext.KEY, default)


from threading import Thread


def start_thread(request, func, *args, **kwargs):
    current_context = request.req_context.get_context()

    return Thread(target=lambda: request.req_context.call_with_context(current_context, func, *args, **kwargs))


class Request:
    def __init__(self):
        self.req_context = RequestContext()

request = Request()


def handle_skill_call(*args, **kwargs):
    QUEUE.put(f"{get_ident()}, I handle skill call with {request.req_context.get_context()}")

    request.req_context.call_with_context("Context 2", handle_catalog_call)
    
    QUEUE.put(f"{get_ident()}, I finish your call {request.req_context.get_context()}")

    
def handle_catalog_call(*arg, **kwargs):
    parent_context = request.req_context.get_context()

    QUEUE.put(f"{get_ident()}, I handle catalog call with {request.req_context.get_context()}")
    
    new_thread_1 = start_thread(request, handle_external_service_call, request, "Service 1")
    new_thread_2 = start_thread(request, handle_external_service_call, request, "Service 2", True)
    new_thread_3 = start_thread(request, handle_external_service_call, request, "Service 3")
    
    request.req_context.call_with_context("Context 3", handle_sth_task)

    new_thread_1.start()
    new_thread_2.start()
    new_thread_3.start()
    
    new_thread_1.join()
    new_thread_2.join()
    new_thread_3.join()

    request.req_context.call_with_context("Context 5", handle_sth_task)


def handle_external_service_call(request, service_name, start_a_thread=False, *arg, **kwargs):
    QUEUE.put(f"{get_ident()}, I handle external service call to {service_name} with {request.req_context.get_context()}")

    if start_a_thread:
        start_thread(request, handle_sth_task)
    else:
        request.req_context.call_with_context("Context 4", handle_sth_task)


def handle_sth_task(*args, **kwargs):
    QUEUE.put(f"{get_ident()}, I handle some tasks with {request.req_context.get_context()}")


request.req_context.call_with_context("Context 1", handle_skill_call)

list(QUEUE.queue)

["4710768128, [{'X_RAY_CONTEXT': 'Context 1'}]",
 '4710768128, I handle skill call with Context 1',
 "4710768128, [{'X_RAY_CONTEXT': 'Context 1'}, {'X_RAY_CONTEXT': 'Context 2'}]",
 "4710768128, [{'X_RAY_CONTEXT': 'Context 1'}, {'X_RAY_CONTEXT': 'Context 2'}]",
 '4710768128, I handle catalog call with Context 2',
 "4710768128, [{'X_RAY_CONTEXT': 'Context 1'}, {'X_RAY_CONTEXT': 'Context 2'}]",
 "4710768128, [{'X_RAY_CONTEXT': 'Context 1'}, {'X_RAY_CONTEXT': 'Context 2'}]",
 "4710768128, [{'X_RAY_CONTEXT': 'Context 1'}, {'X_RAY_CONTEXT': 'Context 2'}]",
 "4710768128, [{'X_RAY_CONTEXT': 'Context 1'}, {'X_RAY_CONTEXT': 'Context 2'}, {'X_RAY_CONTEXT': 'Context 3'}]",
 '4710768128, I handle some tasks with Context 3',
 "123145395781632, [{'X_RAY_CONTEXT': 'Context 2'}]",
 '123145395781632, I handle external service call to Service 1 with Context 2',
 "123145395781632, [{'X_RAY_CONTEXT': 'Context 2'}, {'X_RAY_CONTEXT': 'Context 4'}]",
 '123145395781632, I handle some tasks with Context 4',
 "